In [87]:
#################################################################
# Load data
#################################################################

import os
os.environ["NFL_HOME"] = "/home/sam/repos/hobby-repos/nfl/"

from common.data_loader import DataLoader

# Get raw data
loader = DataLoader()
games_df, plays_df, players_df, location_data_df = loader.get_data(weeks=[week for week in range (1,10)])

print(location_data_df.head(10))


       gameId  playId    nflId     displayName  frameId    frameType  \
0  2022091200      64  35459.0  Kareem Jackson        1  BEFORE_SNAP   
1  2022091200      64  35459.0  Kareem Jackson        2  BEFORE_SNAP   
2  2022091200      64  35459.0  Kareem Jackson        3  BEFORE_SNAP   
3  2022091200      64  35459.0  Kareem Jackson        4  BEFORE_SNAP   
4  2022091200      64  35459.0  Kareem Jackson        5  BEFORE_SNAP   
5  2022091200      64  35459.0  Kareem Jackson        6  BEFORE_SNAP   
6  2022091200      64  35459.0  Kareem Jackson        7  BEFORE_SNAP   
7  2022091200      64  35459.0  Kareem Jackson        8  BEFORE_SNAP   
8  2022091200      64  35459.0  Kareem Jackson        9  BEFORE_SNAP   
9  2022091200      64  35459.0  Kareem Jackson       10  BEFORE_SNAP   

                    time  jerseyNumber club playDirection      x      y     s  \
0  2022-09-13 00:16:03.5          22.0  DEN         right  51.06  28.55  0.72   
1  2022-09-13 00:16:03.6          22.0  DEN  

In [105]:
#################################################################
# Filter down plays df to situations we are interested in
#################################################################
filtered_plays_df = plays_df.copy()

# Find the starting plays
print("Filtering data...")
original_play_length = len(filtered_plays_df)
print(f'Total plays: {original_play_length}')

# Filter out penalties
filtered_plays_df = filtered_plays_df[filtered_plays_df['playNullifiedByPenalty'] == 'N']
# Filter out rows with 'PENALTY' in the 'playDescription' column
filtered_plays_df = filtered_plays_df[~filtered_plays_df['playDescription'].str.contains("PENALTY", na=False)]
print(f'Total plays after filtering out penalties: {len(filtered_plays_df)}')

# Filter down to valid Man or Zone defensive play calls
filtered_plays_df = filtered_plays_df[filtered_plays_df['pff_manZone'].isin(['Man', 'Zone'])]
print(f'Total plays after filtering to valid Man or Zone classifications: {len(filtered_plays_df)}')

# Filter for only rows that indicate a pass play
filtered_plays_df = filtered_plays_df[filtered_plays_df['passResult'].notna()]
print(f'Total plays after filtering to only pass plays: {len(filtered_plays_df)}')

# Filter for only plays where the win probablity isn't lopsided (between 0.2 and 0.8)
filtered_plays_df = filtered_plays_df[(filtered_plays_df['preSnapHomeTeamWinProbability'] > 0.1) & (filtered_plays_df['preSnapHomeTeamWinProbability'] < 0.9)]
print(f'Total plays after filtering out garbage time: {len(filtered_plays_df)}')

# Filter for only third down or fourth down plays
filtered_plays_df = filtered_plays_df[filtered_plays_df['down'].isin([3, 4])]
print(f'Total plays after filtering for 3rd or 4th down: {len(filtered_plays_df)}')

# Filter for plays that are in our gameIds (in location data df)
filtered_plays_df = filtered_plays_df[filtered_plays_df['gameId'].isin(location_data_df['gameId'].unique())]
print(f'Total plays after making sure they are in our location data: {len(filtered_plays_df)}')

print(filtered_plays_df.columns)


Filtering data...
Total plays: 16124
Total plays after filtering out penalties: 15908
Total plays after filtering to valid Man or Zone classifications: 14910
Total plays after filtering to only pass plays: 9103
Total plays after filtering out garbage time: 7079
Total plays after filtering for 3rd or 4th down: 2028
Total plays after making sure they are in our location data: 2028
Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore',
       'playNullifiedByPenalty', 'absoluteYardlineNumber',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'expectedPoints', 'offenseFormation', 'receiverAlignment',
       'playClockAtSnap', 'passResult', 'passLength', 'targetX', 'targetY',
       'playAction', 'dropbackType', 'dropbackDistance', 'passLocationType',
       'timeToThrow', 'timeInTackleBox', 'timeT

In [106]:
#################################################################
# Cut down df to only cols we care about
#################################################################
# Cut down to columns we care about
keep_cols_from_plays = ['gameId', 'playId', 'possessionTeam', 'defensiveTeam', 'pff_manZone']
filtered_plays_df = filtered_plays_df.loc[:, keep_cols_from_plays].drop_duplicates()

# Make sure we don't have any NAs in this cut down col df
filtered_plays_df.dropna()
print(f'Total plays after cutting down to our cols and dropping NAs: {len(filtered_plays_df)}')

# Print total plays
print(filtered_plays_df.head())

Total plays after cutting down to our cols and dropping NAs: 2028
        gameId  playId possessionTeam defensiveTeam pff_manZone
9   2022100204    1422            DAL           WAS        Zone
10  2022100912     445            DAL            LA        Zone
16  2022100913    2604            CIN           BAL        Zone
20  2022092505    3437             NE           BAL         Man
30  2022091803    1222            IND           JAX         Man


In [107]:
#################################################################
# Create merged df that has gameId, playId, frameID all before SNAP, with x, y, and offense/defense
#################################################################
import pandas as pd
import numpy as np

# Create a copy of the location tracking data, cut it down to columns we care about
loc_trimmed_df = location_data_df.copy()
keep_cols  = [
    'gameId',
    'playId',
    'nflId',
    'frameId',
    'frameType',
    'club',
    'x',
    'y',
]
loc_trimmed_df = location_data_df.loc[:, keep_cols]

# Cut down location tracking data copy to only before the snap and where the team isn't valid
loc_trimmed_df = loc_trimmed_df[(loc_trimmed_df["frameType"] == "BEFORE_SNAP") & (loc_trimmed_df["club"] != "football")]

print(loc_trimmed_df.head())

       gameId  playId    nflId  frameId    frameType club      x      y
0  2022091200      64  35459.0        1  BEFORE_SNAP  DEN  51.06  28.55
1  2022091200      64  35459.0        2  BEFORE_SNAP  DEN  51.13  28.57
2  2022091200      64  35459.0        3  BEFORE_SNAP  DEN  51.20  28.59
3  2022091200      64  35459.0        4  BEFORE_SNAP  DEN  51.26  28.62
4  2022091200      64  35459.0        5  BEFORE_SNAP  DEN  51.32  28.65


In [108]:
#################################################################
# Label offense/defense, cut down df further, and sort
#################################################################
# Merge the two datasets such that we can have the possession and defensive team for each row
merged_df = pd.merge(filtered_plays_df, loc_trimmed_df, on=['gameId', 'playId'], how='inner')

# Tag the "side" of the player for each row (that being "off" or "def")
merged_df['side'] = np.where(merged_df['club'] == merged_df['possessionTeam'], 'off', 'def')

# Drop some columns we don't need anymore
merged_df = merged_df.drop(['possessionTeam', 'defensiveTeam', 'club', 'frameType'], axis=1)

# Sort for deterministic frame ordering
merged_df = merged_df.sort_values(['gameId','playId','frameId'])

# Let's see what we have
print(merged_df.head())

             gameId  playId pff_manZone    nflId  frameId      x      y side
4264920  2022090800     236        Zone  35472.0        1  38.21  28.48  off
4265012  2022090800     236        Zone  38577.0        1  31.07  30.45  def
4265104  2022090800     236        Zone  41239.0        1  31.67  29.14  def
4265196  2022090800     236        Zone  42392.0        1  38.08  29.51  off
4265288  2022090800     236        Zone  42816.0        1  29.85  28.78  def


In [109]:
#################################################################
# Decide the target sequence length using the median number of frames per play
#################################################################
frame_counts = (merged_df
                .groupby(['gameId','playId'])['frameId']
                .nunique())
min_frames = int(np.percentile(frame_counts.values, 10))
print(f"Using plays that have above {min_frames} frames")

Using plays that have above 70 frames


In [110]:
#################################################################
# Built dataset where frame is only min frames and each side has exactly 11 players
#################################################################
def exactly_eleven_per_side(play_df: pd.DataFrame) -> bool:
    return (
        play_df.loc[play_df.side == 'off', 'nflId'].nunique() == 11 and
        play_df.loc[play_df.side == 'def', 'nflId'].nunique() == 11
    )

def slot_order_by_left_to_right(play_df: pd.DataFrame, side: str) -> list:
    side_df = play_df.loc[play_df["side"] == side]
    stats = (side_df
             .groupby('nflId', as_index=True)[['x','y']]
             .median()
             .rename(columns={'x':'x_med','y':'y_med'})
             .sort_values(['x_med','y_med']))
    return stats.index.tolist()  # list of sorted NFL player ids for this play to determine median x --> y player locs

def build_side_xy_cube(play_df: pd.DataFrame, side: str, frames: np.ndarray) -> np.ndarray:
    # pivot to (frames x slots) for x and y, fill missing with NaN, then stack → (min_frames, 11, 2)
    side_df = play_df.loc[play_df["side"] == side]


    # Takes the long df and goes from frameId, slot, x, y as cols to:
    # slot, 0, 1, 2 as cols ... with frameId 1, frameId 2... etc as the rows....shape is (min_frames, 11)
    x_mat = side_df.pivot_table(index='frameId', columns='slot', values='x')
    y_mat = side_df.pivot_table(index='frameId', columns='slot', values='y')


    # It's possible certain players don't have exact tracking data throughout (ie one player has frame 10 and 12 but not frame 11), this will end up breaking our shape and cause issues downstream for model training
    # So this forces the matrix to have for each frame 
    x_mat = x_mat.reindex(index=frames, columns=range(11), fill_value=np.nan)
    y_mat = y_mat.reindex(index=frames, columns=range(11), fill_value=np.nan)

    # Stack along last axis such that we return (min_frames, 11, 2)
    return np.stack([x_mat.to_numpy(), y_mat.to_numpy()], axis=-1)

# Init series maps
off_series = {}
def_series = {}

# Lists to peek at later if we skip plays
skipped_wrong_player_count_list = []   # plays where offense or defense had >11 unique players
skipped_under_min_frames_list = []     # plays with fewer than min_frames

# Iterate on each play
for (game_id, play_id), play in merged_df.groupby(['gameId','playId'], sort=False):
    # Skip if not 11 players
    if not exactly_eleven_per_side(play):
        skipped_wrong_player_count_list.append((game_id, play_id))
        continue

    # Define slot maps (left→right by median x, tie-break median y)
    off_slots = slot_order_by_left_to_right(play, 'off')
    def_slots = slot_order_by_left_to_right(play, 'def')

    # Create a map that goes player id --> index so we can assign each player to an index as we go frame by frame
    off_id2slot = {pid: i for i, pid in enumerate(off_slots)}
    def_id2slot = {pid: i for i, pid in enumerate(def_slots)}

    # Assign slots (if offense use offensive map, if defense, use defensive map)
    tmp = play.copy()
    tmp['slot'] = np.where(
        tmp['side'] == 'off',
        tmp['nflId'].map(off_id2slot),
        tmp['nflId'].map(def_id2slot)
    )

    # Choose frame window (last min_frames frames)
    frames_all = np.sort(tmp['frameId'].unique())
    if frames_all.size < min_frames:
        skipped_under_min_frames_list.append((game_id, play_id))
        continue
    frames = frames_all[-min_frames:]  # Get the last min frames, so each play is consistent

    # Build offense/defense cubes: (min_frames, 11, 2) the 2 is x and y coords
    off_arr = build_side_xy_cube(tmp, 'off', frames)
    def_arr = build_side_xy_cube(tmp, 'def', frames)

    off_series[(game_id, play_id)] = off_arr
    def_series[(game_id, play_id)] = def_arr

print(f"Kept plays: {len(off_series)}")
print(f"Skipped (>11 players): {len(skipped_wrong_player_count_list)}")
print(f"Skipped (<{min_frames} frames): {len(skipped_under_min_frames_list)}")

Kept plays: 1826
Skipped (>11 players): 0
Skipped (<70 frames): 202


In [ ]:
#################################################################
# Data functions to build dataset with imputing and converting to tensors
#################################################################

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

def impute_timewise(X_np: np.ndarray) -> np.ndarray:
    """
    X_np: (T, F) with NaNs.
    Impute per feature (column) along time:
      1) forward-fill if we miss a frame, assume the player stayed where he was last seen.
      2) back-fill
      3) fill remaining NaNs with column mean (0 if all NaN)
    """
    df = pd.DataFrame(X_np)                 # (T, 11 players * 2 features = 44)

    # Copy the last known values fwd in time if there's missing NaNs
    # Fill any leading NaNs that had no earlier data
    # Example: [NaN, 3, 4, NaN, NaN, 7] ---> foward fill [NaN, 3, 4, 4, 4, 7] ---> backward fill [3, 3, 4, 4, 4, 7]
    # If a whole column has NaNs we then fill it with 0s (only time this realistically kicks in)
    df = df.ffill().bfill().fillna(0.0)


    return df.values.astype(np.float32)

def build_plays_data(off_series, def_series, labels_dict):
    X, y = [], []
    for key, off_arr in off_series.items():
        def_arr = def_series[key]
        X_play = np.concatenate([off_arr, def_arr], axis=1).reshape(off_arr.shape[0], -1)
        X_play = impute_timewise(X_play)
        X.append(torch.from_numpy(X_play).float())
        y.append(torch.tensor(labels_dict[key], dtype=torch.long))
    return X, y

In [133]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Build labels dict mapping of (gameId, playId) --> 0/1
label_map = {'Man': 1, 'Zone': 0}
labels_dict = {(r.gameId, r.playId): label_map[r.pff_manZone] for r in filtered_plays_df.itertuples()}

# Get dataset (imputed, converted to tensor)
X, y = build_plays_data(off_series, def_series, labels_dict)

# Splittys
idx_train, idx_val = train_test_split(
    np.arange(len(X)),                  # Create an array from 0 to x number of plays
    test_size=0.2,                      # Choosing standard 20% for test size
    random_state=42,                    # Life universe and everything
    stratify=[yy.item() for yy in y]    # Takes each small tensor (these are small scalar tensors) of our labels, converts to 0 or 1, then makes a list of the labels [0, 0, 1, 0, 1,...] --> says split the data while keeping same ratio of 0s and 1s in both train and validation sets
    )

# Make tensor datasets
# NOTE: X will be of shape (play_count, min_frames, 44)
# NOTE: Y will be of shape (play_count, )
train_ds = TensorDataset(torch.stack([X[i] for i in idx_train]),    # Each x is (min_frame, 44)
                         torch.stack([y[i] for i in idx_train]))
val_ds   = TensorDataset(torch.stack([X[i] for i in idx_val]),
                         torch.stack([y[i] for i in idx_val]))

# Reproducibility seeds
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# For deterministic behavior (slower, optional)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Make dataloaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64)

In [134]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size=44, hidden_size=64, num_layers=1, dropout=0.0, bidir=False, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidir,
        )
        out_dim = hidden_size * (2 if bidir else 1)
        self.head = nn.Sequential(
            nn.LayerNorm(out_dim),
            nn.Linear(out_dim, num_classes)
        )

    def forward(self, x):  # x: (B, T, F)
        out, (h_n, c_n) = self.lstm(x)        # out: (B, T, H)
        last = out[:, -1, :]                   # use last timestep representation
        logits = self.head(last)               # (B, C)
        return logits

In [135]:
from sklearn.utils.class_weight import compute_class_weight

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_size=44, hidden_size=64, num_layers=1, dropout=0.0, bidir=False).to(device)

# Zone dominates class weighting, calc distro then assign man a higher waiting on the CE loss
y_all = np.array([dataset.y[i].item() for i in range(len(dataset))], dtype=int) # Build an array of all dataset labels
y_train = y_all[idx_train] # Slice to the training fold
classes = np.array([0, 1], dtype=int)  # 0=Zone, 1=Man
w = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
print("Class weights (Zone, Man):", w)
class_weights = torch.tensor(w, dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):

    # Train
    model.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = criterion(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Val
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X).argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.numel()
    print(f"Epoch {epoch+1}: val acc = {correct/total:.3f}")


Class weights (Zone, Man): [0.90458488 1.1179173 ]
Epoch 1: val acc = 0.495
Epoch 2: val acc = 0.443
Epoch 3: val acc = 0.456
Epoch 4: val acc = 0.508
Epoch 5: val acc = 0.530
Epoch 6: val acc = 0.527
Epoch 7: val acc = 0.579
Epoch 8: val acc = 0.484
Epoch 9: val acc = 0.470
Epoch 10: val acc = 0.522
Epoch 11: val acc = 0.464
Epoch 12: val acc = 0.546
Epoch 13: val acc = 0.473
Epoch 14: val acc = 0.549
Epoch 15: val acc = 0.470
Epoch 16: val acc = 0.582
Epoch 17: val acc = 0.489
Epoch 18: val acc = 0.464
Epoch 19: val acc = 0.484
Epoch 20: val acc = 0.467
Epoch 21: val acc = 0.568
Epoch 22: val acc = 0.503
Epoch 23: val acc = 0.519
Epoch 24: val acc = 0.574
Epoch 25: val acc = 0.579
Epoch 26: val acc = 0.533
Epoch 27: val acc = 0.536
Epoch 28: val acc = 0.549
Epoch 29: val acc = 0.467
Epoch 30: val acc = 0.459
Epoch 31: val acc = 0.462
Epoch 32: val acc = 0.464
Epoch 33: val acc = 0.522
Epoch 34: val acc = 0.571
Epoch 35: val acc = 0.568
Epoch 36: val acc = 0.464
Epoch 37: val acc = 0.

In [136]:
from sklearn.metrics import classification_report

model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(device), y.to(device)
        pred = model(X).argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(y.cpu().numpy())

print(classification_report(all_true, all_preds, target_names=["Zone", "Man"]))

              precision    recall  f1-score   support

        Zone       0.68      0.36      0.47       202
         Man       0.50      0.79      0.61       164

    accuracy                           0.55       366
   macro avg       0.59      0.57      0.54       366
weighted avg       0.60      0.55      0.53       366

